In [ ]:
from core import *

sys.path.append("/Users/sakthi.n/Documents/Opensource/code-as-data/")

fdep_path = "/Users/sakthi.n/Downloads/tmp"
G = get_graph(fdep_path)

In [ ]:
import pickle
from core import *

sys.path.append("/Users/sakthi.n/Documents/Opensource/code-as-data/")
with open('test.gpickle', 'rb') as f:
    g = pickle.load(f)

In [ ]:
path, path_str = get_path_str(g, "Euler.API.Txns.Endpoints.TranspiledTxns:refundByTxnId","Product.OLTP.Refund.Validation:makeRefund")

In [ ]:
print(path)

In [ ]:
root = "Euler.API.Txns.Endpoints.TranspiledTxns:refundByTxnId"
des = "Product.OLTP.Refund.Validation:makeRefund"

g = get_subgraph(g, root)

print("cnt: ",len(g.nodes()))
for node in g.nodes():
    print(node)

In [ ]:
for nod in g.nodes():
    if g.has_edge(nod,nod):
        g.remove_edge(nod,nod)
    
cycles = list(nx.simple_cycles(g))
print(f"❌ Found {len(cycles)} cycle(s)")
cnt = 0
for cycle in cycles:
    first = cycle[0]
    last = cycle[-1] 
    if g.has_edge(last,first):
        g.remove_edge(last,first)

In [ ]:
ls = get_topo_order(g)


print(len(ls))
for (node,lvl) in ls:
    print(lvl, node)    

In [ ]:
mp = dict() 
for (v, k) in ls:
    if mp.get(k,None):
        mp[k].append(v)
    else:
        mp[k] = [v]

for i in range(0,51):
    print(len(mp[i]))

In [ ]:
get_visualize(g)

In [ ]:
print("Path len:",len(path))

In [ ]:
g = process_graph_parallel(g, response_schema=DetailedMetaData)

In [ ]:
g.nodes[root]

In [ ]:
import json 
from testcase.llm import call_gemini

path,path_str = get_path_str(g, root, des)
coreMetaData = ""
for idx,node in enumerate(path): 
    if idx+1 == len(path):
       break 

    (_ ,metainfo) = all_child_expectone(g, node, path[idx+1])

    prompt = summarize_prompt.format(current_function=node,next_function=path[idx+1],metainfo=metainfo)
    result = call_gemini(user_prompt=prompt,response_format="application/json",response_schema=MetaData)
    dct = json.loads(result)

    coreMetaData += f"FunctionName: {node.split(":")[-1]}\nshortsummary: {dct["behavior_summary"]}\npreconditions: \n{"\n".join(dct["preconditions"])}\nposconditions: \n{"\n".join(dct["postconditions"])}\nsideeffects: \n{"\n".join(dct["side_effects"])}\n"
    state_deps = dct.get("state_dependencies",None)
    if state_deps:
        coreMetaData += f"DB: {state_deps.get("db","None")}\nEnv: {state_deps.get("env","None")}\nRedis: {state_deps.get("redis","None")}\n\n"
    else:
        coreMetaData += "DB: None\nEnv: None\nRedis: None\n\n"

    print(coreMetaData)


In [ ]:
_,info = all_childs(g, des)

prompt = final_prompt.format(function_name=des,metainfo=coreMetaData,currmetainfo=info,api_path=path_str)

res = call_gemini(user_prompt=prompt,response_format="text/plain")
print(res)